In [66]:
import yfinance as yf
import pandas as pd
import matplotlib as plt
import xlsxwriter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook,load_workbook
import datetime

In [67]:

# Get data
stock = yf.Ticker("AAPL")
income_statement = stock.income_stmt
balance_sheet = stock.balance_sheet
cash_flow_statement= stock.cashflow



In [68]:
def auto_fit_columns(sheet):
    """Automatically adjusts column widths to fit cell content."""
    for col in sheet.columns:
        max_length = 0
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except TypeError:
                pass
        adjusted_width = (max_length + 2)  # Adjust width for readability
        sheet.column_dimensions[col[0].column_letter].width = adjusted_width

def convert_to_date(sheet, cells):
    """Converts cell values in `cells` to mm/dd/yyyy format."""
    for cell in cells:
        if isinstance(sheet[cell].value, datetime.datetime):
            sheet[cell].value = sheet[cell].value.date()
            sheet[cell].number_format = 'mm/dd/yyyy'

def swap_rows_in_worksheet(worksheet):
    """Swaps rows from bottom to top (reversing order) in the given worksheet."""
    max_row = worksheet.max_row
    for row_index in range(2, (max_row // 2) + 1):
        top_row_cells = list(worksheet.iter_rows(min_row=row_index, max_row=row_index, values_only=True))[0]
        bottom_row_cells = list(worksheet.iter_rows(min_row=max_row - row_index + 2, max_row=max_row - row_index + 2, values_only=True))[0]
        for col_index, (top_cell_value, bottom_cell_value) in enumerate(zip(top_row_cells, bottom_row_cells), start=1):
            worksheet.cell(row=row_index, column=col_index).value = bottom_cell_value
            worksheet.cell(row=max_row - row_index + 2, column=col_index).value = top_cell_value

In [69]:
# Load the existing workbook
wb = load_workbook('out.xlsx')

# Handle sheet creation/updating dynamically
sheet_names = ['Income Statement', 'Balance Sheet', 'Cashflow Statement']
data_frames = [income_statement, balance_sheet, cash_flow_statement]
for sheet_name, data_frame in zip(sheet_names, data_frames):
    if sheet_name not in wb.sheetnames:  # Create sheet if it doesn't exist
        ws = wb.create_sheet(sheet_name)
        dataframe_to_rows(data_frame).to_excel(ws, startrow=1, startcol=1)
    else:  # Update existing sheet
        ws = wb[sheet_name]
        ws.delete_rows(1, ws.max_row)
        # Append new data
        for row in dataframe_to_rows(data_frame):
            ws.append(row)
# Formatting and modifications
for ws in wb.worksheets:
    auto_fit_columns(ws)
    convert_to_date(ws, ['B1', 'C1', 'D1', 'E1'])
    swap_rows_in_worksheet(ws)  # Uncomment if necessary

In [70]:
input_sheet = wb.create_sheet("Input")

# Put the stock name in the new sheet
input_sheet['A1'] = "Stock Name"
input_sheet['B1'] = stock.info['longName']

In [71]:


# Save the workbook
wb.save('out.xlsx')
